**Troubleshooting Scripts - Transaction Log**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Analyzing VLFs in database transaction log**

You would like to see reasonable number (a few hundreds max) of evenly sized VLF files. 

SQL Server 2016 SP2+. Use DBCC LOGINFO on old version of SQL Server

In [ ]:
SELECT 
    COUNT(*) as [VLF Count]
    ,MIN(vlf_size_mb) as [Min VLF Size (MB)]
    ,MAX(vlf_size_mb) as [Max VLF Size (MB)]
    ,AVG(vlf_size_mb) as [Avg VLF Size (MB)]
FROM sys.dm_db_log_info(DB_ID())
OPTION (RECOMPILE, MAXDOP 1);

In [ ]:
DBCC LOGINFO

**Getting log free space and truncation status for all databases**

In [ ]:
IF OBJECT_ID('tempdb..#SpaceUsed') IS NOT NULL
    DROP TABLE #SpaceUsed;
GO

CREATE TABLE #SpaceUsed
(
    database_id SMALLINT NOT NULL,
    file_id SMALLINT NOT NULL,
    space_used DECIMAL(15,3) NOT NULL,
    PRIMARY KEY(database_id, file_id)
);
 
EXEC master..sp_MSforeachdb 
N'USE[?];
INSERT INTO #SpaceUsed(database_id, file_id, space_used)
    SELECT DB_ID(''?''), file_id,
         (size - CONVERT(INT,FILEPROPERTY(name, ''SpaceUsed''))) / 128.
FROM sys.database_files
WHERE type = 1;';
 
SELECT 
    d.database_id, d.name, d.recovery_model_desc
    ,d.state_desc, d.log_reuse_wait_desc, m.physical_name
    ,m.is_percent_growth
    ,IIF(m.is_percent_growth = 1,m.growth,CONVERT(DECIMAL(15,3),m.growth / 128.0)) AS [Growth (MB or %)]
    ,CONVERT(DECIMAL(15,3),m.size / 128.0) AS [Size (MB)]
    ,IIF(m.max_size = -1,-1,CONVERT(DECIMAL(15,3),m.max_size / 128.0)) AS [Max Size(MB)]
    ,s.space_used as [Space Used(MB)]
FROM 
    sys.databases d WITH (NOLOCK) 
        JOIN sys.master_files m WITH (NOLOCK) ON
        d.database_id = m.database_id
    LEFT OUTER JOIN #SpaceUsed s ON
        s.database_id = m.database_id AND
        s.file_id = m.file_id
ORDER BY 
    d.database_id;

**Get the list of active transactions**

  

You can kill the session that prevents log truncation if needed

In [ ]:
SELECT
    dt.database_id
    ,DB_NAME(dt.database_id) as [DB]
    ,st.session_id
    ,CASE at.transaction_state
        WHEN 0 THEN 'Not Initialized'
        WHEN 1 THEN 'Not Started'
        WHEN 2 THEN 'Active'
        WHEN 3 THEN 'Ended (R/O)'
        WHEN 4 THEN 'Commit Initialize'
        WHEN 5 THEN 'Prepared'
        WHEN 6 THEN 'Committed'
        WHEN 7 THEN 'Rolling Back'
        WHEN 8 THEN 'Rolled Back'
     END AS [State]
    ,at.transaction_begin_time
    ,es.login_name
    ,ec.client_net_address
    ,ec.connect_time
    ,dt.database_transaction_log_bytes_used
    ,dt.database_transaction_log_bytes_reserved
    ,er.status
    ,er.wait_type
    ,er.last_wait_type
    ,sql.text AS [SQL]
FROM
    sys.dm_tran_database_transactions dt WITH (NOLOCK)
        JOIN sys.dm_tran_session_transactions st WITH (NOLOCK) ON
            dt.transaction_id = st.transaction_id
        JOIN sys.dm_tran_active_transactions at WITH (NOLOCK) ON
            dt.transaction_id = at.transaction_id
        JOIN sys.dm_exec_sessions es WITH (NOLOCK) ON
            st.session_id = es.session_id
        JOIN sys.dm_exec_connections ec WITH (NOLOCK) ON
            st.session_id = ec.session_id
        LEFT OUTER JOIN sys.dm_exec_requests er WITH (NOLOCK) ON
            st.session_id = er.session_id
        CROSS APPLY
            sys.dm_exec_sql_text(ec.most_recent_sql_handle) sql
ORDER BY
    dt.database_transaction_begin_time
OPTION (RECOMPILE, MAXDOP 1);